## What version are we?

In [1]:
#!pwsh
#!about

The below script needs to be able to find the current output cell; this is an easy method to get it.

.NET Interactive© 2020 Microsoft CorporationVersion: 1.0.142601+586ab2f29818edea7c19c383874977fe5d0391fcBuild date: 2020-09-17T15:38:08.7508410Zhttps://github.com/dotnet/interactive


In [1]:
#!pwsh
$psversionTable


Name                           Value
----                           -----
PSVersion                      7.0.2
PSEdition                      Core
GitCommitId                    7.0.2
OS                             Microsoft Windows 10.0.18363
Platform                       Win32NT
PSCompatibleVersions           {1.0, 2.0, 3.0, 4.0…}
PSRemotingProtocolVersion      2.3
SerializationVersion           1.1.0.1
WSManStackVersion              3.0


In [1]:
#!pwsh
$host.Name
$host.UI.RawUI

.NET Interactive Host

ForegroundColor       : -1
BackgroundColor       : -1
CursorPosition        : 
WindowPosition        : 
CursorSize            : 
BufferSize            : 100,50
WindowSize            : 100,50
MaxWindowSize         : 100,50
MaxPhysicalWindowSize : 100,50
KeyAvailable          : 
WindowTitle           : 



## Markdown contains formatted text and images 

* *emphasis*
* **Strong Emphasis**
* `code`
* etc. 

```PowerShell
write-host "This is a script block formatted as PowerShell" -fore red
# How good the format is varies ! 
```

Folding markdown cells hides the code cells between them and the next markdown with the same title level. **THIS ONE** breaks folding because the is no code beneath. 

Local images work better if the path is relative to the .ipynb file or they can be fetched over http    
![Untitled2.png](./Untitled2.png)    
![James Avatar](https://jhoneill.github.io/assets/james.jpg)

In [1]:
#!html
<strong>.NET interactive allows Cells to have a <em>language</em> of HTML</strong><br/>  
Language is selected with <em>Magic commands</em> <code>#!html</code>, <code>#!pwsh</code>, 
<code>#!javascript</code>, <code>#!fsharp</code> and <code>#!csharp</code>. <br/>
There is a full list of magic commands further down.

.NET interactive allows Cells to have a language of HTML 
Language is selected with Magic commands #!html , #!pwsh , 
 #!javascript , #!fsharp and #!csharp . 
There is a full list of magic commands further down.

In [1]:
#!markdown
**Bold**

Bold

## Normal shell things ... 

In [1]:
#!pwsh
git status

On branch master
Your branch is up to date with 'origin/master'.

nothing to commit, working tree clean

In [1]:
#!pwsh
write-host -ForegroundColor Green "boo hoo"

boo hoo

## Where were we called from

In [1]:
#!pwsh
$pshome
$p = Get-process -id $pid   
$p, $p.Parent, $p.Parent.Parent,$p.Parent.Parent.parent,$p.Parent.Parent.parent.parent
Get-CimInstance -ClassName win32_process -Filter "processID = $($p.id)" |  select -expand commandline
Get-CimInstance -ClassName win32_process -Filter "processID = $($p.Parent.id)" |  select -expand commandline
Get-CimInstance -ClassName win32_process -Filter "processID = $($p.Parent.parent.id)" |  select -expand commandline
Get-CimInstance -ClassName win32_process -Filter "processID = $($p.Parent.parent.parent.id)" |  select -expand commandline
Get-CimInstance -ClassName win32_process -Filter "processID = $($p.Parent.parent.parent.parent.id)" |  select -expand commandline

C:\Users\P10506111\.dotnet\tools\.store\microsoft.dotnet-interactive\1.0.142601\microsoft.dotnet-interactive\1.0.142601\tools\netcoreapp3.1\any\runtimes\win\lib\netcoreapp3.1

 NPM(K)    PM(M)      WS(M)     CPU(s)      Id  SI ProcessName
 ------    -----      -----     ------      --  -- -----------
    168   145.59     152.47       3.94   99912   1 dotnet-interactive
     23    11.57      22.21       0.34   92832   1 dotnet
     77    56.06      72.69       3.19  103708   1 python
     16     8.98      16.02       0.27  119892   1 python
    134   244.02     118.69   2,514.39    7180   1 explorer
"C:\Users\P10506111\.dotnet\tools\dotnet-interactive.exe" jupyter --default-kernel powershell C:\Users\P10506111\AppData\Roaming\jupyter\runtime\kernel-d43b15dc-b43a-4c17-9850-02f55b53d5d0.json
dotnet interactive jupyter --default-kernel powershell C:\Users\P10506111\AppData\Roaming\jupyter\runtime\kernel-d43b15dc-b43a-4c17-9850-02f55b53d5d0.json
C:\ProgramData\Anaconda3\python.exe C:\Progra

*In a browser you can check who owns the port you connected to*   
**Note: things which display a progress bar and then try to close it can cause an error, so turn progress display off**

In [1]:
#!pwsh
$ProgressPreference = 'silentlyContinue'
Get-NetTCPConnection -LocalPort 9815,8888 -State Listen | ft -a


LocalAddress LocalPort RemoteAddress RemotePort State  AppliedSetting OwningProcess
------------ --------- ------------- ---------- -----  -------------- -------------
::1          8888      ::            0          Listen                103708
127.0.0.1    8888      0.0.0.0       0          Listen                103708


## PowerShell modules loaded in a clean session, and modules available

In [1]:
#!pwsh
Get-module


ModuleType Version    PreRelease Name                                ExportedCommands
---------- -------    ---------- ----                                ----------------
Binary     7.0.0.0               CimCmdlets                          {Get-CimAssociatedInstance, G…
Manifest   7.0.0.0               Microsoft.PowerShell.Management     {Add-Content, Clear-Content, …
Manifest   7.0.0.0               Microsoft.PowerShell.Utility        {Add-Member, Add-Type, Clear-…
Manifest   1.0.0.0               NetTCPIP                            {Find-NetRoute, Get-NetCompar…


In [1]:
#!pwsh
Get-Module -ListAvailable | 
    group -no -Property @{e={
        Split-Path -Parent $_.Path | 
            %{if ([Version]::TryParse((Split-Path $_ -Leaf), [ref]$null)) {Split-Path -Parent $_} else {$_} } | 
                 Split-Path -Parent
     }} | ft -AutoSize -wrap


Count Name
----- ----
    1 C:\Program Files\PowerShell\Modules
   73 C:\Program Files\WindowsPowerShell\Modules
    5 C:\Users\P10506111\.dotnet\tools\.store\microsoft.dotnet-interactive\1.0.142601\microsoft.dot
      net-interactive\1.0.142601\tools\netcoreapp3.1\any\Modules
    8 C:\users\p10506111\.dotnet\tools\.store\microsoft.dotnet-interactive\1.0.142601\microsoft.dot
      net-interactive\1.0.142601\tools\netcoreapp3.1\any\runtimes\win\lib\netcoreapp3.1\Modules
   15 C:\Users\P10506111\Documents\PowerShell\Modules
   50 C:\WINDOWS\system32\WindowsPowerShell\v1.0\Modules


In [1]:
#!pwsh
Get-Module -ListAvailable | where path -Match "netcoreapp" | ft name ,exportedCommands


Name                                    ExportedCommands
----                                    ----------------
Microsoft.DotNet.Interactive.PowerShell {[Enter-AzShell, Enter-AzShell], [New-PlotlyChart, New-Plo…
Microsoft.PowerShell.Archive            {[Compress-Archive, Compress-Archive], [Expand-Archive, Ex…
PackageManagement                       {[Find-Package, Find-Package], [Get-Package, Get-Package],…
PowerShellGet                           {[Find-Command, Find-Command], [Find-DSCResource, Find-DSC…
ThreadJob                               {[Start-ThreadJob, Start-ThreadJob]}
CimCmdlets                              {[Get-CimAssociatedInstance, Get-CimAssociatedInstance], […
Microsoft.PowerShell.Diagnostics        {[Get-WinEvent, Get-WinEvent], [New-WinEvent, New-WinEvent…
Microsoft.PowerShell.Host               {[Start-Transcript, Start-Transcript], [Stop-Transcript, S…
Microsoft.PowerShell.Management         {[Add-Content, Add-Content], [Clear-Content, Clear-Content…
Microsof

## What is in the PowerShell module which comes with .Net Interactive ? 

In [1]:
#!pwsh
ipmo Microsoft.DotNet.Interactive.PowerShell
Get-command -Module Microsoft.DotNet.Interactive.PowerShell


CommandType     Name                                               Version    Source
-----------     ----                                               -------    ------
Cmdlet          Enter-AzShell                                      0.1.0      Microsoft.DotNet.Int…
Cmdlet          New-PlotlyChart                                    0.1.0      Microsoft.DotNet.Int…
Cmdlet          Out-Display                                        0.1.0      Microsoft.DotNet.Int…


## Out-Display, plotly charts and other & preformed HTML
First magic command  `#!time` and use of  `[Microsoft.DotNet.Interactive.Kernel]::HTML($html)` 

**Note:** Plotly uses Java-script and may not render in all viewers 

In [1]:
#!pwsh
#!time 
$x = "Counting" | Out-Display -PassThru
1..5 | %{sleep 1; $x.Update($_) }

5

Wall time: 5099.6544ms

In [1]:
#!pwsh
New-Object -TypeName XPlot.Plotly.Graph+Bar -Property @{
            x = @('C#', 'JavaScript', 'PHP', 'PowerShell','Python')
            y = @(23,17,12,35,29)
} | New-PlotlyChart -Title 'Students' | Out-Display

In [1]:
#!pwsh
$html = Get-Command -Module Microsoft.DotNet.Interactive.PowerShell | 
        ConvertTo-Html -Fragment -Property Name , Version 

[Microsoft.DotNet.Interactive.Kernel]::HTML($html) | Out-Display

Name,Version
Enter-AzShell,0.1.0
New-PlotlyChart,0.1.0
Out-Display,0.1.0


**If you don't have the files for the next code-cell skip it and load the SVG data from the following one**

In [1]:
#!pwsh
Import-Module ~\documents\azdo\serverautomationdemo\utils\PSGraph\ -Force # My version!
. ~\documents\azdo\serverautomationdemo\utils\PlotDevopsPipelines.ps1

$svg = plot_pipeline ~\documents\azdo\serverautomationdemo\CI\FollowOn.pipeline.yml  -DestinationPath "" 
$svg | out-file -Encoding utf8 -path ./demo.svg

**If you used the previous code-cell you can skip the next one**

In [1]:
#!pwsh
$svg = Get-Content ./demo.svg

In [1]:
#!pwsh
[Microsoft.DotNet.Interactive.Kernel]::HTML($svg) | Out-Display

<?xml version="1.0" encoding="UTF-8" standalone="no"?> <!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN" "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd"> <!-- Generated by graphviz version 2.38.0 (20140413.2041) --> <!-- Title: g Pages: 1 --> g cluster841de02f6eb6 Stage: FollowOnStage <!-- FollowOn.pipeline --> FollowOn.pipeline FollowOn.pipeline <!-- Job: ExitStageLeft --> Job: ExitStageLeft Job: ExitStageLeft <!-- FollowOn.pipeline->Job: ExitStageLeft --> FollowOn.pipeline->Job: ExitStageLeft <!-- 841de02f6eb6 --> <!-- DownloadPipelineArtifact --> DownloadPipelineArtifact DownloadPipelineArtifact <!-- Job: ExitStageLeft->DownloadPipelineArtifact --> Job: ExitStageLeft->DownloadPipelineArtifact <!-- PowerShell --> PowerShell PowerShell <!-- DownloadPipelineArtifact->PowerShell --> DownloadPipelineArtifact->PowerShell

## Wrapper for Out-Display

In [1]:
#!pwsh
. .\NotebookOutput.ps1
Out-Html -? | select -ExpandProperty Description


This command has three ways of working. It can 
* Take HTML (via the pipeline or as a parameter) and output it into a cell
* Make an HTML table or list from objects, selecting properties, first, last etc.
* Take a script block to generate html or objects to transform to a list/table



In [1]:
#!pwsh
# Pipe in , convert to table.
Get-command -Module  Microsoft.DotNet.Interactive.PowerShell 
    | Out-HTML -AsTable name,version

Name,Version
Enter-AzShell,0.1.0
New-PlotlyChart,0.1.0
Out-Display,0.1.0


In [1]:
#!pwsh
#Pass script block, format as list
Cell -AsList name,version {
    Get-command -Module  Microsoft.DotNet.Interactive.PowerShell 
}

Name: Enter-AzShell Version: 0.1.0 Name: New-PlotlyChart Version: 0.1.0 Name: Out-Display Version: 0.1.0

In [1]:
#!pwsh
#Relies on earlier cell to set $svg - pass raw output in a variable  
if ($svg) {
    cell $svg
}

<?xml version="1.0" encoding="UTF-8" standalone="no"?><!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN" "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd"><!-- Generated by graphviz version 2.38.0 (20140413.2041) --><!-- Title: g Pages: 1 --> g cluster841de02f6eb6 Stage: FollowOnStage <!-- FollowOn.pipeline --> FollowOn.pipeline FollowOn.pipeline <!-- Job: ExitStageLeft --> Job: ExitStageLeft Job: ExitStageLeft <!-- FollowOn.pipeline->Job: ExitStageLeft --> FollowOn.pipeline->Job: ExitStageLeft <!-- 841de02f6eb6 --><!-- DownloadPipelineArtifact --> DownloadPipelineArtifact DownloadPipelineArtifact <!-- Job: ExitStageLeft->DownloadPipelineArtifact --> Job: ExitStageLeft->DownloadPipelineArtifact <!-- PowerShell --> PowerShell PowerShell <!-- DownloadPipelineArtifact->PowerShell --> DownloadPipelineArtifact->PowerShell

In [1]:
#!pwsh
#If we loaded the grpah module earlier, raw output from a script block.
if (test-path Function:\plot_pipeline) {
    cell  {plot_pipeline ~\documents\azdo\serverautomationdemo\CI\FollowOn.pipeline.yml  -DestinationPath "" }
}

<?xml version="1.0" encoding="UTF-8" standalone="no"?><!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN" "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd"><!-- Generated by graphviz version 2.38.0 (20140413.2041) --><!-- Title: g Pages: 1 --> g clusterc51aaa375c59 Stage: FollowOnStage <!-- FollowOn.pipeline --> FollowOn.pipeline FollowOn.pipeline <!-- Job: ExitStageLeft --> Job: ExitStageLeft Job: ExitStageLeft <!-- FollowOn.pipeline->Job: ExitStageLeft --> FollowOn.pipeline->Job: ExitStageLeft <!-- c51aaa375c59 --><!-- DownloadPipelineArtifact --> DownloadPipelineArtifact DownloadPipelineArtifact <!-- Job: ExitStageLeft->DownloadPipelineArtifact --> Job: ExitStageLeft->DownloadPipelineArtifact <!-- PowerShell --> PowerShell PowerShell <!-- DownloadPipelineArtifact->PowerShell --> DownloadPipelineArtifact->PowerShell

## Doug Finke's Notebook module

In [1]:
#!pwsh
ipmo PowerShellNotebook
$ProgressPreference = 'silentlyContinue'
Get-command -Module PowerShellNotebook | Get-help | ft -a -wrap name,Synopsis


Name                              Synopsis
----                              --------
Add-NotebookCode                  Add-NotebookCode adds PowerShell code to a code block
Add-NotebookMarkdown              Add-NotebookMarkdown adds Markdown to a markdown block
Convert-MarkdownToNoteBook        Convert a markdown file to an interactive PowerShell Notebook
ConvertFrom-NotebookToMarkdown    Take and exiting PowerShell Notebook and convert it to markdown
ConvertTo-PowerShellNoteBook      
ConvertTo-SQLNoteBook             
Export-NotebookToPowerShellScript Exports all code blocks from a PowerShell Notebook to a
                                  PowerShell script
Export-NotebookToSqlScript        Exports all code blocks from a SQL Notebook to a SQL script
Get-Notebook                      Get-Notebook reads the metadata of a single (or folder of)
                                  Jupyter Notebooks
Get-NotebookContent               Get-NotebookContents reads the contents of a Jupyter Note

In [1]:
#!pwsh
cd (split-path $profile -Parent)
Get-Notebook '~\Git tutorial.ipynb'
 
Get-NotebookContent '~\Git tutorial.ipynb' | Select -first 5 | ft 


NoteBookName     : Git tutorial.ipynb
KernelName       : .net-powershell
CodeBlocks       : 39
MarkdownBlocks   : 32
NoteBookFullName : C:\Users\P10506111\Git tutorial.ipynb



NoteBookName       Type     Source
------------       ----     ------
Git tutorial.ipynb markdown Here we are going to look at git as purely local file management.  The…
Git tutorial.ipynb code     cd ~…
Git tutorial.ipynb markdown Lets **Initialize a new local repo**. …
Git tutorial.ipynb code     $null = md dummyProj …
Git tutorial.ipynb markdown Make a file, **add** it to the list of files Git tracks, and make a **…


## Mixed languages, even in a single block, & variable sharing via magic commands
https://github.com/dotnet/interactive/blob/main/docs/variable-sharing.md 

In [1]:
#!pwsh
Write-Output "Hello from PowerShell!"

#!fsharp
let numbers = [0 .. 5]
numbers

Hello from PowerShell!

index,value
0,0
1,1
2,2
3,3
4,4
5,5


In [1]:
#!pwsh
#!share  --from fsharp numbers
$numbers

0
1
2
3
4
5

In [1]:
#!pwsh
#!value --name someJson
{
    "what": "Some Json",
    "why": "To share it!"
}

In [1]:
#!pwsh
#!share someJson --from value
ConvertFrom-Json -InputObject $someJson


what      why
----      ---
Some Json To share it!


## List of Magic commands

In [1]:
#!pwsh
#!lsmagic

.NET kernel #!csharp , #!c# , #!C# Run the code that follows using the csharp kernel. #!fsharp , #!f# , #!F# Run the code that follows using the fsharp kernel. #!pwsh , #!powershell Run the code that follows using the pwsh kernel. #!javascript , #!js Run the code that follows using the javascript kernel. #!html Run the code that follows using the html kernel. #!value Stores a value that can then be shared with other subkernels. #!lsmagic List the available magic commands / directives #!markdown Convert the code that follows from Markdown into HTML #!time Time the execution of the following code in the submission. #!log Enables session logging. #!about Show version and build information #!connect Connects additional subkernels named-pipe Connects to a kernel using named pipes signalr Connects to a kernel using SignalR

csharp kernel #i <null> #r <null> #!whos , #!who Display the names of the current top-level variables and their values. #!share Share a .NET variable between subkernels #!lsmagic List the available magic commands / directives

fsharp kernel #i <null> #r <null> #!whos , #!who Display the names of the current top-level variables and their values. #!share Share a .NET variable between subkernels #!lsmagic List the available magic commands / directives

pwsh kernel #!share Share a .NET variable between subkernels #!lsmagic List the available magic commands / directives

javascript kernel #!lsmagic List the available magic commands / directives

html kernel #!lsmagic List the available magic commands / directives

value kernel #!whos , #!who Display the names of the current top-level variables and their values. #!lsmagic List the available magic commands / directives